In [40]:
# for cpu
# %pip install "qdrant-client[fastembed]"

# for gpu
# %pip install 'qdrant-client[fastembed-gpu]'

In [41]:
from pathlib import Path
import json

product_path = "../training-sets/MEN_FASHION_WITH_REVIEWS"
product_list = json.loads(Path(product_path + "/raw_products.json").read_text())
products = product_list[0:50]


In [42]:
import torch # type: ignore

# from imagebind import data
# from imagebind.models import imagebind_model
# from imagebind.models.imagebind_model import ModalityType


# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"

# # Instantiate model
# model = imagebind_model.imagebind_huge(pretrained=True)
# model.eval()
# model.to(device)

In [43]:
from urllib.parse import urlparse
def _get_product_text_data(product: dict):
    
    description = product.get("description", None)
    revised_description = product.get("description", None)
    name = product.get("name", None)
    product_page_url = product.get("product_page_url", None)
    
    
    if description is None or name is None or product_page_url is None:
        raise ValueError("description, name or product_page_url is missing")
    
    
    parsed_url = urlparse(product_page_url)
    product_url_path = parsed_url.path
    
    text_to_embded = f"{name}\n{description}\n{product_url_path}"
    
    return text_to_embded

In [44]:


from sentence_transformers import SentenceTransformer
def embed_product_text_data(product: dict):
    product_text_data = _get_product_text_data(product)
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(product_text_data)
    return embeddings    
    
# embed_product_text_data(products[0])

In [45]:
images_directory = "../training-sets/MEN_FASHION_WITH_REVIEWS/images/"

In [46]:
from transformers import ViTModel, ViTImageProcessor
from PIL import Image

def embed_product_image(product: dict):
   
    product_asin = product.get("product_asin", None)
        
    if product_asin is None:
        raise ValueError("product_asin is missing")
    
    path_to_image = Path(f"{images_directory}/{product_asin}.png")
    if not path_to_image.exists():
        raise FileNotFoundError(f"{path_to_image} does not exist.")
    

    image = Image.open(path_to_image.absolute())

    model = ViTModel.from_pretrained("google/vit-base-patch16-224")
    image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224") # type: ignore

    # Load and process image
    inputs = image_processor(images=image, return_tensors="pt")

    # Generate embedding
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0].detach().numpy()
    
    return embedding
    
# embed_product_image(products[0])

### Embed the product text data

In [47]:
from qdrant_client import QdrantClient, models

client = QdrantClient(location=":memory:")

encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
client.recreate_collection(
    collection_name="product_text_data",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # type: ignore | Vector size is defined by used model
        distance=models.Distance.COSINE),
)

client.upload_records(
    collection_name="product_text_data",
    records=[
        models.Record(
            id=idx, 
            vector=encoder.encode(_get_product_text_data(product)),  # type: ignore
            payload=product
        ) for idx, product in enumerate(products)
    ]
)


/tmp/ipykernel_1946360/1684884611.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
/tmp/ipykernel_1946360/1684884611.py:13: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  client.upload_records(


### Perform a search on the text

In [48]:
# Let's now search for something

hits = client.search(
    collection_name="product_text_data",
    query_vector=encoder.encode("Do you have any men sneakers I can wear for the winter?"),
    limit=1
)
for hit in hits:
  print("score:", hit.score, hit.payload )

score: 0.45414936542510986 {'id': 'fde9e6c3-6ae2-40c6-b9bf-27c726ebfe9b', 'name': 'Wide Fit Trainers for Men Arch Support Road Running Athletic Sneakers Breathable Gym Shoes for Fitness Jogging Walking - Fresh Core', 'description': "    Product details     Care instructions     Machine Wash       Sole material     Rubber       Outer material     3-Layer Mesh       Inner material     Mesh      About this item   Breathable Knit Upper: The upper of mens trainer is made of mesh and synthetic materials. The knitted fabric is comfortable, breathable and lightweight to keep your feet dry during exercise   Wide Toe Box: These wide trainers are good for wide feet and people suffering from foot problems like plantar fasciitis, foot pain, flat feet, bunions and hammertoes   Slip On Design: Men's slip on trainers with elastic and soft synthetic lining protect your back heel from abrasion, which is convenient to put on and take off   Cushioned Midsole: The EVA midsole provides effective shock absor

### Embed the product images

In [49]:
from qdrant_client import QdrantClient, models

client = QdrantClient(location=":memory:")

encoder = ViTModel.from_pretrained("google/vit-base-patch16-224")


client.recreate_collection(
    collection_name="product_image_data",
    vectors_config=models.VectorParams(
        size=768, # Vector size is defined by used model
        distance=models.Distance.COSINE),
)

client.upload_records(
    collection_name="product_image_data",
    records=[
        models.Record(
            id=idx, 
            vector=embed_product_image(product),  
            payload=product
        ) for idx, product in enumerate(products)
    ]
)


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1946360/2538211480.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', '

### Perform a search on the product image

In [50]:

# Let's now search for something
hits = client.search(
    collection_name="product_image_data",
    query_vector=embed_product_image(product=products[0]),
    limit=1
)
for hit in hits:
  print("score:", hit.score, hit.payload )

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Multivector  is not found in the collection